In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
import glob
from skimage import io
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import skimage
import json
from PIL import Image
from PIL import Image, ImageDraw,ImageFont
import json

In [ ]:
df=pd.read_csv("output_28.csv")   # PSENET Bounding Boxes

In [ ]:
df[' Points']=df[' Points'].apply(lambda x :x.replace('-1','0'))   # Replacing  '-1' with 0  -> -1 was included to remove invalid bounding boxes

In [ ]:
df["point_new"]=df[" Points"].apply(lambda x :eval(x))

In [ ]:
direc=glob.glob("test_image/*")

In [ ]:
df["point_new"]=df["point_new"].apply(lambda points :[[int(points[0][0]),int(points[0][1])]
                                 ,[int(points[1][0]),int(points[1][1])]
                                 ,[int(points[2][0]),int(points[2][1])]
                                 ,[int(points[3][0]),int(points[3][1])]] )

In [ ]:
def selector(x):
    if x[4]<8 or x[5] <8:
        return False
    elif x[4] <45 and x[5] <18:
        return False
    else:
        return True

In [ ]:
df['selected']=df.apply(selector, axis=1)   # Removing Invalid Boxes  (So Small)

In [ ]:
df.reset_index(inplace=True,drop=True)

In [ ]:
img_list=glob.glob("offical_data//test_image//*")
for i in range(len(img_list)):
    img_list[i]=img_list[i].split("\\")[1]

In [ ]:
output_dic={}
for i in range(25555):
    img_name=df.loc[i,"Image_Name"]
    point=df.loc[i,"point_new"]
    label=None
    if img_name not in output_dic:
        output_dic[img_name]=[{'points':point,'label':None}]
    else:
        output_dic[img_name].append({'points':point,'label':None})

In [ ]:
output_dic['test_216.JPEG']=[]
output_dic['test_24.PNG']=[]
output_dic['test_356.PNG']=[]
output_dic['test_36.JPEG']=[]
output_dic['test_495.PNG']=[]

In [ ]:

with open("null_submission_30000.json",'w',encoding="utf-8") as f:
    json.dump(output_dic,f)

## Yolo Submission

In [ ]:
df=pd.read_csv("output_4_points.csv")

In [ ]:
direc=glob.glob('test_image/*')
img_list=list(df.Image_Name.value_counts().index)

In [ ]:
 count=0
 for i in img_list:
     if f"test_image\\{i}.JPEG" in direc:        
         df.loc[df["Image_Name"]==i,'Image_Name']=f"{i}.JPEG"
     elif  f"test_image\\{i}.PNG" in direc:
         df.loc[df["Image_Name"]==i,'Image_Name']=f"{i}.PNG"
     else:
         count+=1

In [ ]:
df['point']=df['point'].apply(lambda x :x.replace('-1','0'))
df['point']=df['point'].apply(lambda x :x.replace('-',''))

In [ ]:
df['point']=df['point'].apply(lambda x :eval(x))

In [ ]:
def pointchecker(points):
    try:
        if points[3][1]:
            return True
    except:
        return False
df['point_check']=df['point'].apply(pointchecker)    # Removing Boxes which has only 3 points.

In [ ]:
df=df[df['point_check']==True]

In [ ]:
df.drop(columns=['Points', 'Labels', 'Image_Height', 'Image_Width', 'point_check'],inplace=True)

In [ ]:
df.reset_index(drop=True,inplace=True)

In [ ]:
output_dic={}
for i in range(26143):
    img_name=df.loc[i,"Image_Name"]
    point=df.loc[i,"point"]
    label=None
    if img_name not in output_dic:
        output_dic[img_name]=[{'points':point,'label':None}]
    else:
        output_dic[img_name].append({'points':point,'label':None})

In [ ]:
with open('null_submission_25000_1.json','w',encoding='utf-8') as infile:
    json.dump(output_dic,infile)

## Non Max Supression on Bounding Boxes of Yolo and PSENET Model

In [ ]:
with open('null_submission_25000_1.json','r+',encoding='utf-8') as infile:
    yolo_dic=json.load(infile)
    
with open('null_submission_30000.json','r+',encoding='utf-8') as infile:
    pse_dic=json.load(infile)

In [ ]:
def non_max_suppression(boxes, overlapThresh):
   if len(boxes) == 0:
      return []

   # if the bounding boxes integers, convert them to floats --
   # this is important since we'll be doing a bunch of divisions
   if boxes.dtype.kind == "i":
      boxes = boxes.astype("float")
#  
   # initialize the list of picked indexes   
   pick = []

   # grab the coordinates of the bounding boxes
   x1 = boxes[:,0]
   y1 = boxes[:,1]
   x2 = boxes[:,2]
   y2 = boxes[:,3]

   # compute the area of the bounding boxes and sort the bounding
   # boxes by the bottom-right y-coordinate of the bounding box
   area = (x2 - x1 + 1) * (y2 - y1 + 1)
   idxs = np.argsort(y2)

   # keep looping while some indexes still remain in the indexes
   # list
   while len(idxs) > 0:
      # grab the last index in the indexes list and add the
      # index value to the list of picked indexes
      last = len(idxs) - 1
      i = idxs[last]
      pick.append(i)

      # find the largest (x, y) coordinates for the start of
      # the bounding box and the smallest (x, y) coordinates
      # for the end of the bounding box
      xx1 = np.maximum(x1[i], x1[idxs[:last]])
      yy1 = np.maximum(y1[i], y1[idxs[:last]])
      xx2 = np.minimum(x2[i], x2[idxs[:last]])
      yy2 = np.minimum(y2[i], y2[idxs[:last]])

      # compute the width and height of the bounding box
      w = np.maximum(0, xx2 - xx1 + 1)
      h = np.maximum(0, yy2 - yy1 + 1)

      # compute the ratio of overlap
      overlap = (w * h) / area[idxs[:last]]

      # delete all indexes from the index list that have
      idxs = np.delete(idxs, np.concatenate(([last],
         np.where(overlap > overlapThresh)[0])))

   # return only the bounding boxes that were picked 
   return boxes[pick].astype("int")

In [ ]:
# code to combine null_submission_25000_1 and null_submission_30000 and apply non max supression on points to create new points

imgs=list(yolo_dic.keys())
non_max_dic={}
for img in imgs:
    arr=[]
    for i in yolo_dic[img]:
        point=i['points']
        x1=point[0][0]
        y1=point[0][1]
        x2=point[2][0]
        y2=point[2][1]
        arr=np.append(arr,(x1,y1,x2,y2))
        
    for i in pse_dic[img]:
        point=i['points']
        x1=point[0][0]
        y1=point[0][1]
        x2=point[2][0]
        y2=point[2][1]
        arr=np.append(arr,(x1,y1,x2,y2))
        
    arr=arr.reshape(-1,4)
    
    new_boxes=non_max_suppression(arr,0.4)
    non_max_dic[img]=[]
    for point in new_boxes:
        non_max_dic[img].append({'points':[[int(point[0]),int(point[1])],[int(point[2]),int(point[1])],[int(point[2]),int(point[3])],[int(point[0]),int(point[3])]],'label':None})

In [ ]:
non_max_dic['test_216.JPEG']=[]

In [ ]:
with open('null_submission_non_max.json','w',encoding='utf-8') as outfile:
    json.dump(non_max_dic,outfile)